<h1 align='center'>  Facial expression Prediction

In [1]:
#Loading all reqiured libraries
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import LeakyReLU
import os


Using TensorFlow backend.
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

### Defining image 

In [2]:
num_classes = 7 #no of classes og images
img_rows,img_cols = 48,48 #Size of image
batch_size = 32

## Retriving train and validation files

In [3]:
train_data_dir = r'C:\Users\hanit\OneDrive\Desktop\Hanitha\CNN\images\train'
validation_data_dir = r'C:\Users\hanit\OneDrive\Desktop\Hanitha\CNN\images\validation'

### Image agumentation of training images

In [4]:
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=30,
                shear_range=0.3,
                zoom_range=0.3,
                width_shift_range=0.4,
                height_shift_range=0.4,
                horizontal_flip=True,
                fill_mode='nearest')

In [5]:

validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    color_mode='grayscale',
                    target_size=(img_rows,img_cols),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True)

Found 28821 images belonging to 7 classes.


In [7]:
validation_generator = validation_datagen.flow_from_directory(
                        validation_data_dir,
                        color_mode='grayscale',
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
    
                        class_mode='categorical',
                        shuffle=True)

Found 7066 images belonging to 7 classes.


# Model building

## Building CNN model

Creating model object

In [ ]:
model = Sequential()

### Convulution Layer1

In [8]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='glorot_uniform',input_shape=(img_rows,img_cols,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='glorot_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

### Convulution Layer2

In [9]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='glorot_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

### Convulution Layer3 & 4

In [10]:
# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='glorot_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='glorot_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

## Fully connected Layer

In [11]:
#FC1
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


#FC2
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

### Output Layer

In [12]:
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

### Creating training model

In [13]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [14]:
# Creating model(.h5) file with minimum loss and  best accuracy

checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

## Setting stopping creteria

In [15]:
## Monitor for validation error, if contiousl 3 ephos not improved stop the training
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

## Monitor for validation error, learning should not go beyand 0.0001
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

### Compiling model

In [16]:
callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [17]:
#Setting total number of samples for traing and validation
nb_train_samples = 28821
nb_validation_samples = 7066

#Setting up how many number of epochs we want
epochs=25

### Fitting model

In [18]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)


Epoch 1/25
900/900 [==============================] - 506s 562ms/step - loss: 2.1388 - accuracy: 0.1942 - val_loss: 1.8154 - val_accuracy: 0.2616

Epoch 00001: val_loss improved from inf to 1.81542, saving model to Emotion_little_vgg.h5
Epoch 2/25
900/900 [==============================] - 462s 513ms/step - loss: 1.8198 - accuracy: 0.2386 - val_loss: 1.7096 - val_accuracy: 0.2771

Epoch 00002: val_loss improved from 1.81542 to 1.70955, saving model to Emotion_little_vgg.h5
Epoch 3/25
900/900 [==============================] - 518s 576ms/step - loss: 1.7927 - accuracy: 0.2541 - val_loss: 1.6905 - val_accuracy: 0.2596

Epoch 00003: val_loss improved from 1.70955 to 1.69050, saving model to Emotion_little_vgg.h5
Epoch 4/25
900/900 [==============================] - 483s 537ms/step - loss: 1.7646 - accuracy: 0.2694 - val_loss: 1.5292 - val_accuracy: 0.3266

Epoch 00004: val_loss improved from 1.69050 to 1.52924, saving model to Emotion_little_vgg.h5
Epoch 5/25
900/900 [===================